# Step 01: Group Analysis Results (Analysis-only Groups)

This notebook submits analysis grouping jobs to Moody's Risk Modeler for **analysis-only groups**.

Analysis-only groups contain ONLY analysis names (no references to other groups). These can be created immediately since all analyses already exist.

**Tasks:**
- Retrieve Grouping batch from Stage_01/Step_03
- Review grouping job configurations
- Submit grouping jobs to Moody's API
- Track job completion status

**Note:** If you have groups that contain OTHER groups (rollup groups), those are handled separately in Step_02_Group_Rollup.ipynb AFTER this step completes.

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

# Flag to track validation state - allows notebook to complete gracefully on failure
validation_failed = False
validation_errors = []

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_01_Group_Analysis_Results.ipynb')

# Display context
ux.header("Group Analysis Results Batch (Analysis-only Groups)")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Grouping Batch

In [ ]:
# Retrieve Grouping batch from Stage_01/Step_03
ux.subheader("Retrieve Grouping Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.GROUPING not in batches:
    raise ValueError(f"Grouping batch not found. Available: {list(batches.keys())}")

grouping_batch_id = int(batches[BatchType.GROUPING])

ux.success(f"Retrieved Grouping batch: ID={grouping_batch_id}")
step.log(f"Retrieved Grouping batch: ID={grouping_batch_id}")

## 3) Review Grouping Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(grouping_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(grouping_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Check entity existence for FINISHED jobs to show what will be resubmitted
from helpers.entity_validator import EntityValidator
from helpers.constants import JobStatus

validator = EntityValidator()

# Categorize jobs by status and entity existence
pending_jobs = []      # INITIATED jobs - will be submitted
skipped_jobs = []      # FINISHED jobs where entity still exists - will be skipped
resubmit_jobs = []     # FINISHED jobs where entity is missing - will be resubmitted
other_jobs = []        # Jobs in other states (SUBMITTED, RUNNING, etc.)

for job in jobs:
    config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
    config_result = execute_query(config_query, (job['job_configuration_id'],))
    config_data = config_result.iloc[0]['job_configuration_data'] if not config_result.empty else {}
    group_name = config_data.get('Group_Name', 'N/A')
    items_count = len(config_data.get('items', []))
    display_name = f"{group_name} ({items_count} analyses)"
    
    if job['status'] == JobStatus.INITIATED:
        pending_jobs.append(display_name)
    elif job['status'] == JobStatus.FINISHED:
        entity_exists = validator.check_entity_exists_for_job(config_data, batch['batch_type'])
        if entity_exists:
            skipped_jobs.append(display_name)
        else:
            resubmit_jobs.append(display_name)
    else:
        other_jobs.append((display_name, job['status']))

# Display categorized job summary
ux.subheader("Job Summary")

if pending_jobs:
    ux.info(f"\n🆕 Jobs to submit ({len(pending_jobs)}):")
    for name in pending_jobs[:10]:
        ux.info(f"  • {name}")
    if len(pending_jobs) > 10:
        ux.info(f"  ... and {len(pending_jobs) - 10} more")

if resubmit_jobs:
    ux.warning(f"\n🔄 Jobs to resubmit - group missing ({len(resubmit_jobs)}):")
    for name in resubmit_jobs[:10]:
        ux.warning(f"  • {name}")
    if len(resubmit_jobs) > 10:
        ux.warning(f"  ... and {len(resubmit_jobs) - 10} more")

if skipped_jobs:
    ux.success(f"\n✓ Jobs to skip - group exists ({len(skipped_jobs)}):")
    for name in skipped_jobs[:10]:
        ux.success(f"  • {name}")
    if len(skipped_jobs) > 10:
        ux.success(f"  ... and {len(skipped_jobs) - 10} more")

if other_jobs:
    ux.info(f"\n⏳ Jobs in progress ({len(other_jobs)}):")
    for name, status in other_jobs[:5]:
        ux.info(f"  • {name} ({status})")

# Summary line
total_to_process = len(pending_jobs) + len(resubmit_jobs)
ux.info(f"\n📊 Summary: {total_to_process} job(s) will be submitted, {len(skipped_jobs)} will be skipped")

# Validate batch before submission
ux.subheader("Validate Batch")
validation_messages = validate_batch(grouping_batch_id)

# Separate warnings from errors - warnings start with "WARN-"
validation_warnings = [m for m in validation_messages if m.startswith("WARN-")]
validation_errors = [m for m in validation_messages if not m.startswith("WARN-")]

if validation_warnings:
    ux.warning("⚠ Validation warnings (submission will proceed):")
    for warning in validation_warnings:
        ux.warning(f"  {warning}")
    step.log(f"Validation warnings: {len(validation_warnings)}")

if validation_errors:
    validation_failed = True
    ux.error("✗ Batch validation failed:")
    for error in validation_errors:
        ux.error(f"  {error}")
    step.log(f"Batch validation failed: {len(validation_errors)} error(s)")
elif not validation_warnings:
    ux.success("✓ Batch validation passed")
    step.log(f"Verified batch: {total_to_process} jobs to submit, {len(skipped_jobs)} to skip")
else:
    ux.success("✓ Batch validation passed (with warnings)")
    step.log(f"Verified batch: {total_to_process} jobs to submit, {len(skipped_jobs)} to skip (with {len(validation_warnings)} warning(s))")

## 4) Submit Grouping Batch to Moody's

In [ ]:
# Submit batch to Moody's API
if validation_failed:
    ux.warning("⏭ Skipping submission due to validation failure")
    result = None
    failed_count = 0
else:
    ux.subheader("Submit Batch to Moody's")

    ux.info("")
    ux.info("Submission Process:")
    ux.info("  - Each job will group the specified analysis results")
    ux.info("  - Analysis names in the group will be resolved to analysis URIs")
    ux.info("  - Grouped results will be created in Moody's Risk Modeler")
    ux.info("  - Jobs will transition to SUBMITTED status")
    ux.info("  - Batch will transition to ACTIVE status")
    ux.info("")

    # Submit
    ux.info("\nSubmitting batch...")

    # Pass step.step_id to associate batch with this step (not the creation step)
    result = submit_batch(grouping_batch_id, IRPClient(), step_id=step.step_id)

    # Display results
    ux.success(f"\nBatch submission completed")
    ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
    ux.info(f"  Status: {result['batch_status']}")

    # Check for errors
    failed_count = len([j for j in result['jobs'] if 'error' in j])
    if failed_count > 0:
        ux.warning(f"\n{failed_count} job(s) failed to submit")
        for job_result in result['jobs']:
            if 'error' in job_result:
                ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

    step.log(f"Batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if validation_failed:
    # Handle validation failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    
    error_message = "\n".join(validation_errors)
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("BATCH VALIDATION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {grouping_batch_id}")
    ux.error(f"\nValidation errors ({len(validation_errors)}):")
    for error in validation_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease fix the validation errors and retry.")

elif failed_count > 0:
    # Handle submission failures
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}" 
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)
    
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("BATCH SUBMISSION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {grouping_batch_id}")
    ux.info(f"Submitted: {result['submitted_jobs']} job(s)")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nFailed jobs:")
    for error in failed_job_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease review the errors and resubmit failed jobs.")

else:
    # Complete the step successfully
    output_data = {
        'batch_id': grouping_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("GROUPING BATCH (ANALYSIS-ONLY) SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} job(s) to Moody's API")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext steps:")
    ux.info("  - Monitor job progress")
    ux.info("  - If you have rollup groups: Run Step_02_Group_Rollup.ipynb")
    ux.info("  - Otherwise: Proceed to Export stage")